# Исследование надёжности заёмщиков

Заказчик — кредитный отдел банка. Нужно разобраться, влияет ли семейное положение и количество детей клиента на факт погашения кредита в срок. Входные данные от банка — статистика о платёжеспособности клиентов.

Результаты исследования будут учтены при построении модели **кредитного скоринга** — специальной системы, которая оценивает способность потенциального заёмщика вернуть кредит банку.

## Шаг 1. Откройте файл с данными и изучите общую информацию<a name="step_1"></a>

In [2]:
import pandas as pd

In [3]:
#Вывод
df = pd.read_csv('/datasets/data.csv')
df.head(10)

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,высшее,0,женат / замужем,0,F,сотрудник,0,253875.639453,покупка жилья
1,1,-4024.803754,36,среднее,1,женат / замужем,0,F,сотрудник,0,112080.014102,приобретение автомобиля
2,0,-5623.422610,33,Среднее,1,женат / замужем,0,M,сотрудник,0,145885.952297,покупка жилья
3,3,-4124.747207,32,среднее,1,женат / замужем,0,M,сотрудник,0,267628.550329,дополнительное образование
4,0,340266.072047,53,среднее,1,гражданский брак,1,F,пенсионер,0,158616.077870,сыграть свадьбу
5,0,-926.185831,27,высшее,0,гражданский брак,1,M,компаньон,0,255763.565419,покупка жилья
6,0,-2879.202052,43,высшее,0,женат / замужем,0,F,компаньон,0,240525.971920,операции с жильем
7,0,-152.779569,50,СРЕДНЕЕ,1,женат / замужем,0,M,сотрудник,0,135823.934197,образование
8,2,-6929.865299,35,ВЫСШЕЕ,0,гражданский брак,1,F,сотрудник,0,95856.832424,на проведение свадьбы
9,0,-2188.756445,41,среднее,1,женат / замужем,0,M,сотрудник,0,144425.938277,покупка жилья для семьи


**Вывод**

Есть отрицательные значения возраста.'Уровень образования клиента' неправильно указан,есть: "среднее" и "ВЫСШЕЕ" .

## Шаг 2. Предобработка данных<a name="step_2"></a>

### Обработка пропусков

In [3]:
df['family_status'].isna().mean()

0.0

Пропуски в "Семейном положении" не обнаружены.

In [4]:
df['purpose'].isna().mean()

0.0

Пропуски в "Целях получения кредитов" не обнаружены.

In [5]:
df['children'].isna().mean()

0.0

Пропуски в "Количестве детей" не обнаружены.

In [6]:
df['children'].value_counts()

 0     14149
 1      4818
 2      2055
 3       330
 20       76
-1        47
 4        41
 5         9
Name: children, dtype: int64

Значение "-1" скорее всего появилось в результате превращения тире в минус, а "20" в результате опечаток.

Исправим обнаруженные опечатки:

In [7]:
df.loc[df.children == -1, 'children'] = 1

In [8]:
df.loc[df.children == 20, 'children'] = 2

Исправлены опечатки в столбце 'children'.

In [9]:
df['debt'].isna().mean()

0.0

Пропуски в "Наличии кредитов" не обнаружены.

In [10]:
df['debt'].value_counts()

0    19784
1     1741
Name: debt, dtype: int64

In [11]:
df['total_income'].isna().mean()

0.10099883855981417

In [12]:
df['total_income_check'] = df['total_income'].isnull()

In [13]:
df['income_type'].value_counts()

сотрудник          11119
компаньон           5085
пенсионер           3856
госслужащий         1459
безработный            2
предприниматель        2
в декрете              1
студент                1
Name: income_type, dtype: int64

In [14]:
df['total_income'].isna().mean()

0.10099883855981417

In [15]:
df['total_income_check'] = df['total_income'].isnull()
df.loc[df['total_income_check'] == True ,:]

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_check
12,0,NaN,65,среднее,1,гражданский брак,1,M,пенсионер,0,NaN,сыграть свадьбу,True
26,0,NaN,41,среднее,1,женат / замужем,0,M,госслужащий,0,NaN,образование,True
29,0,NaN,63,среднее,1,Не женат / не замужем,4,F,пенсионер,0,NaN,строительство жилой недвижимости,True
41,0,NaN,50,среднее,1,женат / замужем,0,F,госслужащий,0,NaN,сделка с подержанным автомобилем,True
55,0,NaN,54,среднее,1,гражданский брак,1,F,пенсионер,1,NaN,сыграть свадьбу,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...
21489,2,NaN,47,Среднее,1,женат / замужем,0,M,компаньон,0,NaN,сделка с автомобилем,True
21495,1,NaN,50,среднее,1,гражданский брак,1,F,сотрудник,0,NaN,свадьба,True
21497,0,NaN,48,ВЫСШЕЕ,0,женат / замужем,0,F,компаньон,0,NaN,строительство недвижимости,True
21502,1,NaN,42,среднее,1,женат / замужем,0,F,сотрудник,0,NaN,строительство жилой недвижимости,True


Удалим пропуски.

In [38]:
total_income_mean = df['total_income'].mean()

In [36]:
df['total_income'].isna().mean()

0.0

Пропуски в "Доходы" после удаления не обнаружены.

**Вывод**

Проверил все пропуски,удалил которые есть.

### Замена типа данных

In [ ]:
df.info()

Используем функцию 'astype', т.к. столбцы у нас имеют чистый формат 'float64', т.е. перевод в 'int' будет безопасным.

In [ ]:
df['total_income'] = df['total_income'].astype('int')

In [ ]:
df.info()

**Вывод**

Изменил тип данных 'ежемесячный доход' на целочисленнный.

### Обработка дубликатов

Найдем дубликаты

In [5]:
df.duplicated().sum()

71

Есть 54 дублирующие строки. Удалим повторяющиеся значения
Причина:ввод данных заявления несколько раз

In [4]:
df['education'] = df['education'].str.lower()

In [6]:
df=df.drop_duplicates().reset_index(drop=True)

In [7]:
df.duplicated().sum()

0

In [ ]:
df.head(10)

**Вывод**

Удалены дубликаты.

### Лемматизация

Проведем лемматизацию текста в "цель получения кредита".

In [ ]:
#Изучим столбец "цель получения кредита"
df['purpose'].unique()

Имеется много объектов, имеющих схожий смысл, но по-разному записанные.

Воспользуемся модулем pymystem3 и лемматизируем полученный выше список.

In [ ]:
from pymystem3 import Mystem
m = Mystem()

In [ ]:
purpose_dict = df['purpose'].unique()

In [ ]:
lemmas_dict_temp = []

In [ ]:
for purpose in purpose_dict:
    lemmas = m.lemmatize(purpose)
    for l in lemmas:
        lemmas_dict_temp.append(l)

In [ ]:
#Цели кредита
print(lemmas_dict_temp)

In [ ]:
def class_purpose(data):
    data = m.lemmatize(data)
    if 'недвижимость' in data:
        return 'недвижимость'
    if 'жилье' in data:
        return 'недвижимость'
    if 'свадьба' in data:
        return 'свадьба'
    if 'автомобиль' in data:
        return 'автомобиль'
    if 'образование,'  in data:
        return 'образование'
    if  'образование' in data:
        return 'образование'
   
    return 'другое'

In [ ]:
df['purpose_type'] = df['purpose'].apply(class_purpose)

In [ ]:
df['purpose_type'].value_counts()

**Вывод**

Наиболее частые категории взятия кредита: недвижимость, автомобиль, образование, свадьба.

### Категоризация данных

Сформулируем принцип классификации:

Выполним категоризация данных по наличию детей.

In [ ]:
#Класификация по наличию детей
df.groupby('children')['debt'].agg(['count','mean'])

Выполним категоризация данных по заработку клиентов.

In [ ]:
# Сделаем класификация по заработку клиентов
def income_group(income):
     
        if income <= 50000:
                return 'низкий'
        if income <= 100000 and income >= 50001 :
                return 'средний'
        if income <= 160000 and income >= 100001 :
                return 'выше среднего'
        return 'высокий'

In [ ]:
df['total_income_id'] = df['total_income'].apply(income_group)

In [ ]:
df.groupby('total_income_id')['debt'].agg(['count','mean'])

Выполним категоризация данных по семейному статусу.

In [ ]:
#Класификация по семейному статусу
df['family_status'].value_counts()

In [ ]:
df.groupby('family_status')['debt'].agg(['count','mean'])

Выполним категоризация данных по причине взятия кредита.

In [ ]:
#Класификация по причине взятия кредита
df.groupby('purpose_type')['debt'].agg(['count','mean'])

**Вывод**

Категоризировали датафрейм по следующим данным:
по количеству детей ('children');
по уровню зарплаты ('salary');
по семейному положению('family_status');
по причине взятия кредита('purpose_type');


## Шаг 3. Ответьте на вопросы

- Есть ли зависимость между наличием детей и возвратом кредита в срок?

**Вывод**

Больше всего просрочек от семьи с двумя детьми.

- Есть ли зависимость между семейным положением и возвратом кредита в срок?

**Вывод**

Самыми ответственными клиентами банка оказались вдовцы/вдовы,большие задолжности имеют не женатые/не замужние клиенты 

- Есть ли зависимость между уровнем дохода и возвратом кредита в срок?

**Вывод**

Меньше всего должников среди клиентов с низкими доходами,больше всегодолжников среди клиентов с доходом выше среднего 

- Как разные цели кредита влияют на его возврат в срок?

**Вывод**

Много должников,которые берут кредит на операции с автомобилем

## Шаг 4. Общий вывод

Возврат кредита зависит от многих факторов.Все зависимости по категориям клиентов отличаются не на много.

## Чек-лист готовности проекта

Поставьте 'x' в выполненных пунктах. Далее нажмите Shift+Enter.

- [x]  открыт файл;
- [x]  файл изучен;
- [x]  определены пропущенные значения;
- [x]  заполнены пропущенные значения;
- [x]  есть пояснение, какие пропущенные значения обнаружены;
- [x]  описаны возможные причины появления пропусков в данных;
- [x]  объяснено, по какому принципу заполнены пропуски;
- [x]  заменен вещественный тип данных на целочисленный;
- [x]  есть пояснение, какой метод используется для изменения типа данных и почему;
- [x]  удалены дубликаты;
- [x]  есть пояснение, какой метод используется для поиска и удаления дубликатов;
- [x]  описаны возможные причины появления дубликатов в данных;
- [x]  выделены леммы в значениях столбца с целями получения кредита;
- [x]  описан процесс лемматизации;
- [x]  данные категоризированы;
- [x]  есть объяснение принципа категоризации данных;
- [x]  есть ответ на вопрос: "Есть ли зависимость между наличием детей и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между семейным положением и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Есть ли зависимость между уровнем дохода и возвратом кредита в срок?";
- [x]  есть ответ на вопрос: "Как разные цели кредита влияют на его возврат в срок?";
- [x]  в каждом этапе есть выводы;
- [x]  есть общий вывод.